# This notebook is very much work in progress with different attempts and bits to be integrated together. 

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import astropy.units as u

import warnings

from adler.objectdata.AdlerPlanetoid import AdlerPlanetoid

from adler.science.AvgMagModel import AvgMagModel
from adler.science.PhaseCurve import PhaseCurve

from adler.objectdata.Observations import Observations
from adler.objectdata.MPCORB import MPCORB
from adler.objectdata.SSObject import SSObject
from adler.objectdata.AdlerData import AdlerData, AdlerSourceFlags, VALID_AVG_MAG_MODELS, VALID_PHASE_MODELS
from adler.objectdata.objectdata_utilities import get_data_table, get_from_table, mpc_file_preprocessing
import time

In [2]:
plot_filter_colors_white_background = {'u': '#1600ea', 'g': '#31de1f', 'r': '#b52626', 'i': '#370201', 'z': '#ba52ff', 'y': '#61a2b3'}
plot_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}
plot_linestyles = {'u': '--', 'g': (0, (3, 1, 1, 1)), 'r': '-.', 'i': '-', 'z': (0, (3, 1, 1, 1, 1, 1)), 'y': ':'}

In [3]:
# root_filepath = "/home/astro-sobrien/"
# root_filepath = "/Volumes/astro-sobrien/home/astro-sobrien/"
# root_filepath = "/Users/seanobrien/Documents/Adler/"
current_path = os.getcwd()
root_filepath = current_path.split('lsst-adler')[0]

# mpc_test_db_filename = f"{root_filepath}lsst-adler/tests/data/mpc_obs_sbn_testing_database.sqlite"
# rubin_sql_filename_oct = f"{root_filepath}rubin_251002.sqlite"
# rubin_sql_filename_nov = f"{root_filepath}rubin_251105.sqlite"
# rubin_sql_filename_nov2 = f"{root_filepath}rubin_251111.sqlite"
# rubin_sql_filename_nov3 = f"{root_filepath}rubin_251115.sqlite"

input_sql_file = f"{root_filepath}lsst-adler/tests/data/mpc_obs_sbn_testing_database.sqlite"
schema = 'MPC'

# input_sql_file = f"{root_filepath}lsst-adler/tests/data/testing_database.db"
# schema = 'dp03_catalogs_10yr'
input_conn = sqlite3.connect(input_sql_file)
input_cur = input_conn.cursor()

In [4]:
import logging
from astropy.time import Time
log_timestamp = Time.now().isot.replace(":", '_')

# --- Reset logging system (important for Jupyter) ---
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# --- Configure root logger to log only to a file ---
logging.basicConfig(
    filename=f"{root_filepath}/adler_test_logs/adler_test_{log_timestamp}.log",
    filemode='w',
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.DEBUG  # capture INFO, DEBUG, etc.
)

# Optional confirmation
logging.getLogger().info(f"Root logging configured to write to {root_filepath}adler_test{log_timestamp}.log")

logger = logging.getLogger(__name__)

# Attempt to consolidate things

In [5]:

from tqdm import tqdm
import adler.utilities.science_utilities as sci_utils
import math


# Set column to use for magnitude (we don't currently have reduced_mag populated for MPC file format)
mag_col = 'reduced_mag'
magErr_col = 'magErr'

# Set filter list (only ugri present currently, very few u, but keeping in for completeness)
# filter_list=['u', 'g', 'r', 'i', 'z', 'y']
filter_list=['g', 'r']

make_plots=True

if schema=='MPC':
    min_obstime_mjd = math.floor(pd.read_sql_query(f"SELECT MIN(mjd_tai) FROM obs_sbn", input_conn).iloc[0].values[0])
    max_obstime_mjd = math.ceil(pd.read_sql_query(f"SELECT MAX(mjd_tai) FROM obs_sbn", input_conn).iloc[0].values[0])
elif schema=="dp03_catalogs_10yr":
    min_obstime_mjd = math.floor(pd.read_sql_query(f"SELECT MIN(midPointMjdTai) FROM diaSource", input_conn).iloc[0].values[0])
    max_obstime_mjd = math.ceil(pd.read_sql_query(f"SELECT MAX(midPointMjdTai) FROM diaSource", input_conn).iloc[0].values[0])
else:
    print("Schema not recognised")
    raise ValueError("Schema not recognised")

print(min_obstime_mjd)
print(max_obstime_mjd)

from astropy.stats import sigma_clip as astropy_sigma_clip
sig_clip_val = 3

# Set thresholds for magnitude changes
# diff_cuts_arr = np.array([1, 2, 3])
# std_cuts_arr = np.array([5, 6])
diff_cut = 1.5
std_cut = 5

# Defines how many nights of data to retrieve in total
# By default we take the previous 30 nights and the previous 7 nights to allow different checks for outliers
data_timespan_arr = np.array([30, 7])

# Defines how many nights to consider as "new observations" allowing for the checking of "sustained outliers/outbursts"
n_new_nights_arr = np.array([1,3])

60787
60800


In [6]:
#TODO figure out how this feeds in and how to functionise it

# #Defined the strings that will be used as column headers and populated later
# night_mag_string = "outlier_{}night_mag"
# night_std_string = "outlier_{}night_sigma"

# outlier_cols_list = []
# for n_new_nights in n_new_nights_arr:
#     outlier_cols_list.append(night_mag_string.format(n_new_nights))
#     outlier_cols_list.append(night_std_string.format(n_new_nights))

# #TODO include this in function for initialising the output Adler DB
# sql_outlier_cols=""
# for col in outlier_cols_list:
#     sql_outlier_cols += f", {col} INTEGER"

# adler_flags_create_sql = f"CREATE TABLE AdlerSourceFlags(ssObjectId TEXT, filter_name TEXT, diaSourceId TEXT, midPointMjdTai INTEGER{sql_outlier_cols})"

In [7]:
logger.info("----------------------------------------")
logger.info("New loop started")
logger.info("----------------------------------------")

if schema=="MPC":
    process_mjd_arr = np.array([60799.5])
    model_name = "median"
elif schema=="dp03_catalogs_10yr":
    process_mjd_arr = np.array([61590.5])
    model_name = "HG12_Pen16"
else:
    raise ValueError("Schema not recognised")

In [ ]:
# for process_mjd in process_mjd_arr:
process_mjd = process_mjd_arr[0]
start_of_night_mjd = process_mjd - 1

if schema=='MPC':
    obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE mjd_tai BETWEEN '{start_of_night_mjd}' AND '{process_mjd}'", input_conn)
    unique_obj_ids = obj_df.provid.to_numpy()
elif schema=="dp03_catalogs_10yr":
    obj_df = pd.read_sql_query(f"SELECT DISTINCT ssObjectId FROM diaSource WHERE midPointMjdTai BETWEEN '{start_of_night_mjd}' AND '{process_mjd}'", input_conn)
    unique_obj_ids = obj_df.ssObjectId.to_numpy()
else:
    print("Schema not recognised")
    raise ValueError("Schema not recognised")

logger.info(f"{len(unique_obj_ids)} objects to analyze")

if len(unique_obj_ids)==0:
    logger.info(f"No objects to process for {process_mjd}")
    raise ValueError(f"No objects to process for {process_mjd}")
    # continue

output_dir = f"outputs_consolidated_{schema}_{process_mjd}"

os.makedirs(output_dir, exist_ok=True)

output_db = f"{output_dir}/adler_output.sqlite"

# Remove the output DB if it exists
# TODO probably remove this once testing done?
if os.path.exists(output_db):
    os.remove(output_db)

# Shouldn't need this if I implement AdlerSourceFlags and give it its own _get_database_connection, then by default it can write to the supplied path or create the DB if needed
# adler_output_filename = f"{output_dir}/adler_output.sqlite"
# conn_adler_out = sqlite3.connect(adler_output_filename)

for obj_id in tqdm(unique_obj_ids, desc = f"Objects to process for {process_mjd}"):
    for data_timespan in data_timespan_arr:
        planetoid = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssObjectId=obj_id,
                                                                sql_filename=input_sql_file,
                                                                filter_list=filter_list,
                                                                date_range=[process_mjd-data_timespan, process_mjd])

        for filt in planetoid.filter_list:
            df_obs = sci_utils.get_df_obs_filt(planetoid, filt=filt)

            # Currently this will write a row and then update it as AdlerData is designed to overwrite if there's conflict on ssObjectId
            
            # Populate summary AdlerData params for this filter
            # TODO should we take these values or specific values from df_obs_old when considering the exact models (i.e. the 30d_3d, 7d_1d, etc)
            # TODO plan is to write to a metadata section of adler data and this is what we'll write out to the DB
            # Info this will contain is effectively the relevant info to be able to reproduce the given outlier detections, e.g. timespans (t0, t1), model used etc or phase angle min/range if using phase curve
            
            ad_params={}
            ad_params["phaseAngle_min"] = np.amin(df_obs["phaseAngle"])  # * u.deg
            ad_params["phaseAngle_range"] = np.ptp(df_obs["phaseAngle"])  # * u.deg
            ad_params["arc"] = np.ptp(df_obs["midPointMjdTai"])  # * u.d
            ad_params["nobs"] = len(df_obs)
            ad_params["modelFitMjd"] = Time.now().mjd

            err_flag = df_obs.magErr.isnull().all()
            if err_flag:
                logger.info("All magErr values are NaNs, proceed with caution")
            else:
                # Remove observations with large errorbars
                magErr_mask = sci_utils.large_magErr_mask(df_obs)
                df_obs = df_obs[magErr_mask]

            for n_new_nights in n_new_nights_arr:
                # Split into previous observations and observations from the most recent night(s)
                df_obs_old, df_obs_new, *_ = sci_utils.split_obs(df_obs, process_mjd=process_mjd, n_new_nights=n_new_nights)
                logger.info("Previous observations (date < {}): {}".format(process_mjd-n_new_nights, len(df_obs_old)))
                logger.info("New observations ({} <= date < {}): {}".format(process_mjd-n_new_nights, process_mjd, len(df_obs_new)))
                if len(df_obs_old)<2:
                    logger.info("Insufficient number of previous observations, continuing to next band/object")
                    continue
                if len(df_obs_new)==0:
                    logger.info(f"No new observations in {filt}, continuing to next band/object")
                    continue
                
                # Sigma clip old observations
                sig_clip_mask = astropy_sigma_clip(df_obs_old[mag_col], sigma=sig_clip_val).mask
                df_obs_old = df_obs_old[~sig_clip_mask].copy()
                if len(df_obs_old)<2:
                    logger.info("Insufficient number of previous observations after sigma clipping, continuing to next band/object")
                    continue
                
                # Fit model
                if model_name in VALID_AVG_MAG_MODELS:
                    model = AvgMagModel().InitModelObs(mag=df_obs_old[mag_col], magErr=df_obs_old[magErr_col], model_name=model_name)
                    ad_params.update(model.__dict__) # store model values in ad_params
                    planetoid.AdlerData.populate_avg_mag_parameters(filt, **ad_params)

                    res = np.array(df_obs_new[mag_col]) - model.avg_mag
                elif model_name in VALID_PHASE_MODELS:
                    # TODO once SSObject is populated we can take the absolute magnitude from there as the initial guess (this is not currently the case for the MPC obs_sbn file)
                    model = PhaseCurve(H = np.amin(df_obs_old['reduced_mag']) * u.mag, model_name=model_name)
                    pc_fit = model.FitModel(phase_angle=np.array(df_obs_old['phaseAngle']) * u.deg, reduced_mag=np.array(df_obs_old['reduced_mag']) * u.mag, mag_err=np.array([df_obs_old[magErr_col]]) * u.mag)
                    model = model.InitModelSbpy(pc_fit)
                    ad_params.update(model.__dict__) # store model values in ad_params
                    planetoid.AdlerData.populate_phase_parameters(filt, **ad_params)

                    res = np.array(df_obs_new[mag_col]) - model.ReducedMag(df_obs_new['phaseAngle'])
                else:
                    logger.error(f"Model '{model_name}' not recognised")
                    raise ValueError(f"Model '{model_name}' not recognised")
                
                planetoid.AdlerData.write_row_to_database(output_db)
                
                # TODO individual point outliers checks
                # diff_cut
                diff_cut_outlier_arr = sci_utils.outlier_diff(res, diff_cut=diff_cut)
                # TODO column name (dynamic for different timespans/new nights)
                col = "tmp_outlier_col"
                df_obs_new[col] = np.zeros(shape=len(df_obs_new), dtype=float)
                df_obs_new.loc[diff_cut_outlier_arr, col] = res[diff_cut_outlier_arr]

                # sigma_diff_cut

                # Write out res value if above thresholds (only need to check lowest threshold if writing out res value?)

                # TODO sustained checks
                # diff_cut

                # sigma_diff_cut

                # Write out difference in new and old median
                # TODO figure out where to write that

                # TODO figure out how to be writing out 30d_3d, 30d_1d, 7d_3d, 7d_1d cleanly

                # TODO optional plotting routine (function)

# TODO summary stats function

Objects to process for 60799.5: 100%|██████████| 5/5 [00:00<00:00, 18.09it/s]


In [14]:
planetoid.AdlerData.__dict__

{'ssObjectId': '2025 MX40Fake',
 'filter_list': ['g', 'r'],
 'filter_dependent_values': [FilterDependentAdler(filter_name='g', phaseAngle_min=np.float64(10.4569), phaseAngle_range=np.float64(1.5752000000000006), nobs=38, arc=np.float64(6.017512604172225), model_list=['median'], model_dependent_values=[AvgMagModelDependentAdler(filter_name='g', model_name='median', avg_mag=np.float64(19.905139516804923), std_mag=np.float64(0.1293180601923424), modelFitMjd=np.float64(61026.69968279596))]),
  FilterDependentAdler(filter_name='r', phaseAngle_min=np.float64(10.4703), phaseAngle_range=np.float64(1.5581999999999994), nobs=66, arc=np.float64(5.955095509263629), model_list=['median'], model_dependent_values=[AvgMagModelDependentAdler(filter_name='r', model_name='median', avg_mag=np.float64(19.371997673855667), std_mag=np.float64(0.14717178763567726), modelFitMjd=np.float64(61026.69968286432))])]}

In [10]:
planetoid.AdlerSourceFlags

AdlerSourceFlags(ssObjectId='2025 MX40Fake', filter_list=['g', 'r'])

In [12]:
df_obs_new

,ssObjectId,filter_name,diaSourceId,mag,magErr,midPointMjdTai,ra,dec,phaseAngle,topocentricDist,...,heliocentricY,heliocentricZ,topocentricX,topocentricY,topocentricZ,eclipticLambda,eclipticBeta,reduced_mag,num_obs,tmp_outlier_col
22,2025 MX40Fake,r,LorDdACC0000Gugo010000qGX,22.432,0.171,60797.055184,185.456812,6.225557,11.5007,2.385915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000237,66,0.000000
23,2025 MX40Fake,r,LorDdACC0000Gugo010000qGY,22.472,0.169,60797.060023,185.456247,6.225517,11.5020,2.385957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.040199,66,0.000000
24,2025 MX40Fake,r,LorDdACC0000Gugo010000qGZ,22.434,0.150,60797.061476,185.456053,6.225486,11.5024,2.385970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.002187,66,0.000000
25,2025 MX40Fake,r,LorDdACC0000Gugo010000qGa,22.535,0.183,60797.062450,185.455965,6.225491,11.5026,2.385978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.103179,66,0.000000
26,2025 MX40Fake,r,LorDdACC0000Gugo010000qGb,22.476,0.158,60797.063412,185.455828,6.225474,11.5029,2.385986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.044172,66,0.000000
27,2025 MX40Fake,r,LorDdACC0000Gugo010000qGc,22.386,0.162,60797.064858,185.455641,6.225509,11.5032,2.385999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.954160,66,0.000000
28,2025 MX40Fake,r,LorDdACC0000Gugo010000qGd,22.412,0.156,60797.065812,185.455573,6.225508,11.5035,2.386007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.980153,66,0.000000
29,2025 MX40Fake,r,LorDdACC0000Gugo010000qGe,22.392,0.151,60797.066294,185.455545,6.225481,11.5036,2.386011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.960149,66,0.000000
30,2025 MX40Fake,r,LorDdACC0000Gugo010000qGf,22.716,0.189,60797.067287,185.455422,6.225520,11.5039,2.386020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.284141,66,0.000000
31,2025 MX40Fake,r,LorDdACC0000Gugo010000qGg,22.378,0.153,60797.068249,185.455220,6.225504,11.5041,2.386028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.946133,66,0.000000


In [11]:
res

array([-1.37176044, -1.33179897, -1.36981055, -1.2688183 , -1.32782597,
       -1.4178375 , -1.39184511, -1.41184895, -1.08785687, -1.42586454,
       -1.43987687, -1.59688832, -1.43089599, -1.66090368, -1.39790756,
       -1.28691518, -1.33692294, -1.47292682, -1.44693471, -1.34794263,
       -1.23896211, -1.46496601, -1.79406919, -1.6331332 , -1.67437848,
       -1.57241916, -1.67844   , -1.44944837, -1.51446042, -1.70248216,
       -1.53749022, -1.65256361, -1.45060501, -1.52664552, -1.51965356,
       -1.46467419, -1.34968639, -1.37473642, -1.51775693, -1.35776923,
       -1.35278983, -1.35579791])

In [37]:
diff_cut_outlier_arr

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False,  True, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True, False,
       False, False,  True, False, False, False])

In [38]:
col = "tmp_outlier_col"
df_obs_new[col] = np.zeros(shape=len(df_obs_new), dtype=float)
df_obs_new.loc[diff_cut_outlier_arr, col] = res[diff_cut_outlier_arr]

In [39]:
df_obs_new

,ssObjectId,filter_name,diaSourceId,mag,magErr,midPointMjdTai,ra,dec,phaseAngle,topocentricDist,...,heliocentricY,heliocentricZ,topocentricX,topocentricY,topocentricZ,eclipticLambda,eclipticBeta,reduced_mag,num_obs,tmp_outlier_col
22,2025 MX40Fake,r,LorDdACC0000Gugo010000qGX,22.432,0.171,60797.055184,185.456812,6.225557,11.5007,2.385915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.000237,66,0.000000
23,2025 MX40Fake,r,LorDdACC0000Gugo010000qGY,22.472,0.169,60797.060023,185.456247,6.225517,11.5020,2.385957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.040199,66,0.000000
24,2025 MX40Fake,r,LorDdACC0000Gugo010000qGZ,22.434,0.150,60797.061476,185.456053,6.225486,11.5024,2.385970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.002187,66,0.000000
25,2025 MX40Fake,r,LorDdACC0000Gugo010000qGa,22.535,0.183,60797.062450,185.455965,6.225491,11.5026,2.385978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.103179,66,0.000000
26,2025 MX40Fake,r,LorDdACC0000Gugo010000qGb,22.476,0.158,60797.063412,185.455828,6.225474,11.5029,2.385986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.044172,66,0.000000
27,2025 MX40Fake,r,LorDdACC0000Gugo010000qGc,22.386,0.162,60797.064858,185.455641,6.225509,11.5032,2.385999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.954160,66,0.000000
28,2025 MX40Fake,r,LorDdACC0000Gugo010000qGd,22.412,0.156,60797.065812,185.455573,6.225508,11.5035,2.386007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.980153,66,0.000000
29,2025 MX40Fake,r,LorDdACC0000Gugo010000qGe,22.392,0.151,60797.066294,185.455545,6.225481,11.5036,2.386011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.960149,66,0.000000
30,2025 MX40Fake,r,LorDdACC0000Gugo010000qGf,22.716,0.189,60797.067287,185.455422,6.225520,11.5039,2.386020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.284141,66,0.000000
31,2025 MX40Fake,r,LorDdACC0000Gugo010000qGg,22.378,0.153,60797.068249,185.455220,6.225504,11.5041,2.386028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.946133,66,0.000000


In [ ]:
# Contain this in some AdlerSourceFlags object that writes out to database

In [10]:
adler_data.__dict__

{'ssObjectId': '2025 MX40Fake',
 'filter_list': ['g', 'r', 'i'],
 'filter_dependent_values': [FilterDependentAdler(filter_name='g', phaseAngle_min=np.float64(10.4569), phaseAngle_range=np.float64(1.5752000000000006), nobs=38, arc=np.float64(6.017512604172225), model_list=['median'], model_dependent_values=[AvgMagModelDependentAdler(filter_name='g', model_name='median', avg_mag=np.float64(19.905139516804923), std_mag=np.float64(0.1293180601923424), modelFitMjd=np.float64(61024.92295578505))]),
  FilterDependentAdler(filter_name='r', phaseAngle_min=np.float64(10.4703), phaseAngle_range=np.float64(1.5581999999999994), nobs=66, arc=np.float64(5.955095509263629), model_list=['median'], model_dependent_values=[AvgMagModelDependentAdler(filter_name='r', model_name='median', avg_mag=np.float64(19.371997673855667), std_mag=np.float64(0.14717178763567726), modelFitMjd=np.float64(61024.922955821734))]),
  FilterDependentAdler(filter_name='i', phaseAngle_min=nan, phaseAngle_range=nan, nobs=0, arc=

In [11]:
model.__dict__

{'avg_mag': np.float64(19.371997673855667),
 'std_mag': np.float64(0.14717178763567726),
 'model_name': 'median'}

In [12]:
adler_data.write_row_to_database(f"{output_dir}/adler_output.sqlite")

In [64]:
ad_params={}

In [ ]:
ad_params

In [22]:
planetoid.__dict__

{'ssObjectId': '2025 MX40Fake',
 'filter_list': ['g', 'r', 'i'],
 'date_range': [np.float64(60792.5), np.float64(60799.5)],
 'observations_by_filter': [Observations(ssObjectId='2025 MX40Fake', filter_name='g', diaSourceId=array(['LorDdACC0000Gugo010000qFb', 'LorDdACC0000Gugo010000qFc',
         'LorDdACC0000Gugo010000qFd', 'LorDdACC0000Gugo010000qFe',
         'LorDdACC0000Gugo010000qFf', 'LorDdACC0000Gugo010000qFg',
         'LorDdACC0000Gugo010000qFh', 'LorDdACC0000Gugo010000qFi',
         'LorDdACC0000Gugo010000qFq', 'LorDdACC0000Gugo010000qFr',
         'LorDdACC0000Gugo010000qFs', 'LorDdACC0000Gugo010000qFt',
         'LorDdACC0000Gugo010000qFu', 'LorDdACC0000Gugo010000qFv',
         'LorDdACC0000Gugo010000qGB', 'LorDdACC0000Gugo010000qGC',
         'LorDdACC0000Gugo010000qGD', 'LorDdACC0000Gugo010000qGE',
         'LorDdACC0000Gugo010000qGF', 'LorDdACC0000Gugo010000qGG',
         'LorDdACC0000Gugo010000qGH', 'LorDdACC0000Gugo010000qGt',
         'LorDdACC0000Gugo010000qGu', 'LorD

In [15]:
model.__dict__

{'avg_mag': np.float64(19.371997673855667),
 'std_mag': np.float64(0.14717178763567726),
 'model_name': 'median'}

In [67]:
pc = PhaseCurve(model_name="HG12_Pen16")

In [69]:
pc.ReducedMag([1,2,3])

array([19.92389298, 22.31244414, 28.3772531 ])

<HG12_Pen16(H=17.02613933 mag, G12=1.)>

In [51]:
pc.__dict__

{'H': 18,
 'phase_parameter_1': 0.2,
 'phase_parameter_2': None,
 'model_name': 'HG12_Pen16',
 'H_err': None,
 'phase_parameter_1_err': None,
 'phase_parameter_2_err': None,
 'model_function': <HG12_Pen16(H=18., G12=0.2)>}

In [31]:
1/df_obs[magErr_col]

0     5.181347
2     4.672897
3     4.975124
4     6.024096
5     5.617978
6     5.434783
7     6.711409
8     5.617978
9     5.917160
10    4.784689
11    4.950495
12    5.681818
13    6.211180
14    5.847953
Name: magErr, dtype: float64

In [24]:
PhaseCurve(model_name="HG12_Pen16").FitModel(phase_angle=df_obs['phaseAngle'], reduced_mag=df_obs['reduced_mag'], mag_err=df_obs[magErr_col])

/Users/seanobrien/anaconda3/envs/adler_plot/lib/python3.12/site-packages/sbpy/photometry/iau.py:351: RuntimeWarning: divide by zero encountered in log10
  func = -2.5 * np.log10(func)


NonFiniteValueError: Objective function has encountered a non-finite value, this will cause the fit to fail!
This can be caused by non-finite values in the input data or weights, which can be removed with fit(..., filter_non_finite=True), or by diverging model parameters that yield non-finite model values.

In [26]:
df_obs[['phaseAngle', 'reduced_mag', 'magErr']]

,phaseAngle,reduced_mag,magErr
0,11.4921,17.571498,0.193
2,11.4927,17.764479,0.214
3,11.4930,17.698471,0.201
4,11.4945,17.686425,0.166
5,11.4946,17.748421,0.178
6,11.4949,17.813413,0.184
7,11.4951,17.626406,0.149
8,11.4966,17.598363,0.178
9,11.4975,17.691336,0.169
10,11.4977,17.862328,0.209


In [ ]:
planetoid.__dict__

In [ ]:
adler_data

In [ ]:
adler_flags

In [ ]:
# TODO Implement different time spans so its 30d_3d, 30d_1d, 7d_3d, 7d_1d combinations considered

In [ ]:
unique_obj_ids

In [ ]:
obj_id = '2025 MX40'
planetoid = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssObjectId=obj_id,
                                                            sql_filename=input_sql_file,
                                                            filter_list=filter_list,
                                                            date_range=[process_mjd-data_timespan, process_mjd])

adler_data = AdlerData(obj_id, planetoid.filter_list)

filt='r'
df_obs = sci_utils.get_df_obs_filt(planetoid, filt=filt)

In [ ]:
model = AvgMagModel().InitModelObs(mag=df_obs.reduced_mag)

In [ ]:
model.__dict__

# Expanded format:

TODO new format:

(run for night in night but once scripted it will be provided with a given night)

include flag for MPC vs DP0.3 (for testing)

for object in objects:
    for filt in filters:
        for n_new_nights in [1,3,7]:
            sigma clip old observations, calculate median
            check for 1/2/3 mag outliers
            check for 3/4/5 sigma outliers


calculate summary stats;
N objects with outliers
N outliers
distribution of outliers per object and per filter

other things to consider:
consecutive outliers trigger in sigma space as an additional check? (i.e. are 95% of the new points outliers)
are the outliers consistent across multiple filters
minimum number of data points (different for different checks)
minimum number of nights (different for different checks)
if path to database that exists is provided then populate AdlerData and outliers (from AdlerSourceFlags) during processing and write to/update this database


In [ ]:


# Number of days of data to retrieve (i.e. previous 30 nights)
data_timespan=30

from tqdm import tqdm
import adler.utilities.science_utilities as sci_utils
import math


# Set column to use for magnitude (we don't currently have reduced_mag populated for MPC file format)
mag_col = 'reduced_mag'
magErr_col = 'magErr'

# Set filter list (only ugri present currently, very few u, but keeping in for completeness)
filter_list=['u', 'g', 'r', 'i', 'z', 'y']
# filter_list=['g', 'r', 'i']

make_plots=True

if schema=='MPC':
    min_obstime_mjd = math.floor(pd.read_sql_query(f"SELECT MIN(mjd_tai) FROM obs_sbn", input_conn).iloc[0].values[0])
    max_obstime_mjd = math.ceil(pd.read_sql_query(f"SELECT MAX(mjd_tai) FROM obs_sbn", input_conn).iloc[0].values[0])
elif schema=="dp03_catalogs_10yr":
    min_obstime_mjd = math.floor(pd.read_sql_query(f"SELECT MIN(midPointMjdTai) FROM diaSource", input_conn).iloc[0].values[0])
    max_obstime_mjd = math.ceil(pd.read_sql_query(f"SELECT MAX(midPointMjdTai) FROM diaSource", input_conn).iloc[0].values[0])
else:
    print("Schema not recognised")

In [ ]:
from astropy.stats import sigma_clip as astropy_sigma_clip
sig_clip_val = 3

In [ ]:
# Set thresholds for magnitude changes
diff_cuts_arr = np.array([1, 2, 3])
# std_cuts_arr = np.array([3, 4, 5])
std_cuts_arr = np.array([5, 6])

# Defines how many nights to consider as "new observations," allowing for the checking of "sustained outliers/outbursts"
n_new_nights_arr = np.array([1,3,7])

In [ ]:
#Defined the strings that will be used as column headers and populated later
night_mag_string = "outlier_{}night_mag"
night_std_string = "outlier_{}night_sigma"

outlier_cols_list = []
for n_new_nights in n_new_nights_arr:
    outlier_cols_list.append(night_mag_string.format(n_new_nights))
    outlier_cols_list.append(night_std_string.format(n_new_nights))

#TODO include this in function for initialising the output Adler DB
sql_outlier_cols=""
for col in outlier_cols_list:
    sql_outlier_cols += f", {col} INTEGER"

adler_flags_create_sql = f"CREATE TABLE AdlerSourceFlags(ssObjectId TEXT, filter_name TEXT, diaSourceId TEXT, midPointMjdTai INTEGER{sql_outlier_cols})"

In [ ]:
logger.info("----------------------------------------")
logger.info("New loop started")
logger.info("----------------------------------------")

# process_mjd_arr = np.arange(min_obstime_mjd-0.5, max_obstime_mjd+1.5, 1)
# process_mjd_arr = np.arange(60795.5, 60799.5, 1)
process_mjd_arr = np.array([60799.5])

for process_mjd in process_mjd_arr:
    process_date = mjd_to_utc(process_mjd)
    start_of_night = mjd_to_utc(process_mjd - 1)

    # Get list of objects with observations from most recent night preceding the process date
    # obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE obstime BETWEEN '{start_of_night}' AND '{process_date}' LIMIT 20", input_conn)
    obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE obstime BETWEEN '{start_of_night}' AND '{process_date}'", input_conn)
    unique_obj_ids = obj_df.provid.to_numpy()
    logger.info(f"{len(unique_obj_ids)} objects to analyze")

    if len(unique_obj_ids)==0:
        logger.info(f"No objects to process for {process_mjd}")
        continue

    output_dir = f"outputs_lesscols_update_full_{process_mjd}"

    os.makedirs(output_dir, exist_ok=True)

    adler_output_filename = f"{output_dir}/adler_output.sqlite"
    conn_adler_out = sqlite3.connect(adler_output_filename)
    
    #TODO tidy this up into function
    cur_adler_out = conn_adler_out.cursor()

    cur_adler_out.execute("DROP TABLE IF EXISTS AdlerData;")
    cur_adler_out.execute("DROP TABLE IF EXISTS  AdlerSourceFlags;")

    cur_adler_out.execute("CREATE TABLE AdlerData(ssObjectId TEXT, timestamp REAL, PRIMARY KEY (ssObjectId))")
    # added creation of AdlerSourceFlags table
    cur_adler_out.execute(adler_flags_create_sql)

    for obj_id in tqdm(unique_obj_ids, desc = f"Objects to process for {process_mjd}"):
        #Taking all data, no time constraint
        planetoid = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssObjectId=obj_id,
                                                            sql_filename=input_rubin_sql_file,
                                                            filter_list=filter_list,
                                                            date_range=[process_mjd-data_timespan, process_mjd])

        adler_data = AdlerData(obj_id, planetoid.filter_list)

        for filt in planetoid.filter_list:
            df_obs = sci_utils.get_df_obs_filt(planetoid, filt=filt)
            
            #Initialise outlier columns in df_obs
            #TODO Consider how we'll load in previous flags if an existing Adler DB is provided
            for col in outlier_cols_list:
                #Populate outlier columns with zeros
                df_obs[col] = np.zeros(shape=len(df_obs), dtype=int)
            
            #TODO this may change as no longer checking how many datapoints from previous process
            nobs_nomask = len(df_obs)

            err_flag = df_obs.magErr.isnull().all()
            if err_flag:
                logger.info("All magErr values are NaNs, proceed with caution")
            else:
                # Remove observations with large errorbars
                magErr_percentile_cut = 95    #Value (between 0 and 100) to define the percentile above which we cut data with large magErr values
                magErr_mask = (df_obs.magErr <= np.nanpercentile(df_obs.magErr, q=magErr_percentile_cut))
                df_obs = df_obs[magErr_mask]
            
            for n_new_nights in n_new_nights_arr:
                #Split into previous observations and observations from the most recent night(s)
                mask = (df_obs['midPointMjdTai'] < process_mjd-n_new_nights)

                df_obs_old = df_obs[mask].copy()
                df_obs_new = df_obs[~mask].copy()
                logger.info("Previous observations (date < {}): {}".format(process_mjd-1, len(df_obs_old)))
                logger.info("New observations ({} <= date < {}): {}".format(process_mjd-1, process_mjd, len(df_obs_new)))

                if len(df_obs_old)<2:
                    # Taken from adler_demo.py
                    logger.info("Insufficient number of previous observations, continuing to next band/object")
                    continue
                if len(df_obs_new)==0:
                    logger.info(f"No new observations in {filt}, continuing to next band/object")
                    continue

                #TODO consider how this affects writing out to AdlerData (possibly fine with how it's already setup)
                sig_clip_mask = astropy_sigma_clip(df_obs_old[mag_col], sigma=sig_clip_val).mask

                df_obs_old = df_obs_old[~sig_clip_mask].copy()

                if len(df_obs_old)<2:
                    # Taken from adler_demo.py
                    logger.info("Insufficient number of previous observations after sigma clipping, continuing to next band/object")
                    continue

                median_mag = np.median(df_obs_old[mag_col])
                #TODO consider storing relevant res values
                res = np.array(df_obs_new[mag_col]) - median_mag
                
                for diff_cut in diff_cuts_arr:
                    diff_cut_outlier_arr = sci_utils.outlier_diff(res, diff_cut=diff_cut)
                    # Populate outlier rows with True returned by outlier_diff with the current diff_cut value
                    df_obs_new.loc[diff_cut_outlier_arr, night_mag_string.format(n_new_nights)] = diff_cut
                
                std_cut_outlier_arr = [False] * len(df_obs_new)
                if len(df_obs_old)<4:
                    logger.info("Insufficient number of previous observations to check with outlier_sigma_diff")
                    #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
                else:
                    if err_flag:
                        logger.info(f"No measurement errors, can't attempt outlier_sigma_diff")
                        #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
                    else:
                        for std_cut in std_cuts_arr:
                            std_cut_outlier_arr = sci_utils.outlier_sigma_diff(res, df_obs_new[magErr_col], std_sigma=std_cut)
                            df_obs_new.loc[std_cut_outlier_arr, night_std_string.format(n_new_nights)] = std_cut
                
                if make_plots:
                    # Identify any outliers detected
                    tmp_master_outlier_flag = (diff_cut_outlier_arr) | (std_cut_outlier_arr)
                    fig, ax = plt.subplots()
                    ax.errorbar(df_obs_old['midPointMjdTai'], df_obs_old[mag_col], df_obs_old[magErr_col], ls='', marker='.', color='k', label="Previous observations")
                    ax.errorbar(df_obs_new['midPointMjdTai'], df_obs_new[mag_col], df_obs_new[magErr_col], ls='', marker='.', color='c', label="New observations")
                    ax.errorbar(df_obs_new[tmp_master_outlier_flag]['midPointMjdTai'], df_obs_new[tmp_master_outlier_flag][mag_col], df_obs_new[tmp_master_outlier_flag][magErr_col], ls='', marker='x', color='b', label="Outliers")
                    ax.axhline(median_mag)
                    ax.invert_yaxis()
                    ax.set_xlabel("Time [MJD]")
                    ax.set_ylabel(f"{filt}-band Reduced Magnitude")
                    fig.savefig(f"{output_dir}/{obj_id}_{filt}_{n_new_nights}nights_outliers.png", bbox_inches='tight', pad_inches=0.05)
                    plt.close(fig)

                #Use pandas.DataFrame.update to add the outlier flags to the df_obs DataFrame
                df_obs.update(df_obs_new)

            # Identify rows that contain at least 1 outlier flag set to a value greater than 0
            master_outlier_flag = df_obs[outlier_cols_list].any(axis=1)
            df_obs.loc[master_outlier_flag,['ssObjectId', 'filter_name', 'diaSourceId', 'midPointMjdTai'] + outlier_cols_list].to_sql('AdlerSourceFlags', con=conn_adler_out, if_exists='append', index=False)
            
            #TODO consider how to include check and flag for if outliers span multiple consecutive nights
            #Perhaps this is easier now we update df_obs
            #Some kind of resetting cumulative sum to test for sustained outliers that then looks for any value above say 3. [0,1,0,1,2,0] gives [0,1,0,1,3,0]

            #Write nobs to AdlerData
            #TODO write median and standard deviation of previous observations to AdlerData
            #Create AdlerData structure to do the above where we have a particular model that is designed for the median
            adler_data.populate_phase_parameters(filt, **{'nobs':nobs_nomask})
            adler_data.write_row_to_database(adler_output_filename)

            logger.info(f"New information for {obj_id} written to AdlerSourceFlags and AdlerData tables")

    #TODO make summary stats a function
    logger.info(f"Computing summary statistics for {process_mjd}")
    
    cur_adler_out.execute("SELECT COUNT(*) FROM AdlerData")
    n_obj_analzyed = cur_adler_out.fetchall()[0][0]
    logger.info(f"Number of objects analysed: {n_obj_analzyed}")

    # Get column information to check what filters have previously been analysed (and therefore have {filter}_nobs columns in AdlerData)
    cur_adler_out.execute("PRAGMA table_info(AdlerData);")
    adler_out_cols = [row[1] for row in cur_adler_out.fetchall()]
    filter_nobs_columns = [c for c in adler_out_cols if c.endswith('_nobs')]
    current_adlerdata_filters = [c.replace('_nobs', '') for c in filter_nobs_columns]

    # #Manually removing u filter because it has so few observations in comparison
    # filter_nobs_columns.remove('u_nobs')

    filter_columns_sql = ', '.join(filter_nobs_columns)
    nobs_df = pd.read_sql_query(f"SELECT ssObjectId, {filter_columns_sql} FROM AdlerData", conn_adler_out)

    for band in current_adlerdata_filters:
        med_obs_band = np.median(nobs_df[f"{band}_nobs"])
        std_obs_band = np.std(nobs_df[f"{band}_nobs"])
        sum_obs_band = np.sum(nobs_df[f"{band}_nobs"])
        
        logger.info(f"Total number of observations for {band}-band: {sum_obs_band}")
        logger.info(f"Observations per object for {band}-band: Median={med_obs_band}, standard deviation={std_obs_band}")
        

    filters_to_plot = ['g', 'r', 'i']
    fig, ax= plt.subplots()
    ax.hist((nobs_df['g_nobs'], nobs_df['r_nobs'], nobs_df['i_nobs']),
            color=[plot_filter_colors_white_background[filt] for filt in filters_to_plot],
            label=[f"{filt}-band" for filt in filters_to_plot])

    ax.set_xlabel("Observations per filter")
    ax.set_ylabel("Number of objects")
    ax.legend()
    fig.savefig(f"{output_dir}/obs_per_band_hists.png", bbox_inches='tight', pad_inches=0.05)
    plt.close(fig)
    logger.info(f"Histograms of observations per band per object saved to {output_dir}/obs_per_band_hists.png")
    
    for n_new_nights in n_new_nights_arr:
        logger.info(f"Checking for number of outliers when considering last {n_new_nights} night(s) as new observations")
        for diff_cut in diff_cuts_arr:
            cur_adler_out.execute(f"SELECT COUNT(*) FROM AdlerSourceFlags WHERE {night_mag_string.format(n_new_nights)}>={diff_cut}")
            logger.info(f"Number of outliers above {diff_cut} magnitude: {cur_adler_out.fetchall()[0][0]}")
        for std_cut in std_cuts_arr:
            cur_adler_out.execute(f"SELECT COUNT(*) FROM AdlerSourceFlags WHERE {night_std_string.format(n_new_nights)}>={std_cut}")
            logger.info(f"Number of outliers above {std_cut}-sigma: {cur_adler_out.fetchall()[0][0]}")

    #TODO perhaps add example query to find the outliers in the output database

    #TODO distribution of outliers per object and per filter

### Interactive plotting

In [ ]:
#Setup
# obj_id = "2025 MV10"
obj_id = "2025 MX40"
process_mjd = 60799.5
filt = 'g'
n_new_nights = 3

#######

planetoid = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssObjectId=obj_id,
                                                            sql_filename=input_rubin_sql_file,
                                                            filter_list=filter_list,
                                                            date_range=[process_mjd-data_timespan, process_mjd])

adler_data = AdlerData(obj_id, planetoid.filter_list)

df_obs = sci_utils.get_df_obs_filt(planetoid, filt=filt)
            
#Initialise outlier columns in df_obs
#TODO Consider how we'll load in previous flags if an existing Adler DB is provided
for col in outlier_cols_list:
    #Populate outlier columns with zeros
    df_obs[col] = np.zeros(shape=len(df_obs), dtype=int)

nobs_nomask = len(df_obs)

err_flag = df_obs.magErr.isnull().all()
if err_flag:
    logger.info("All magErr values are NaNs, proceed with caution")
else:
    # Remove observations with large errorbars
    magErr_percentile_cut = 95    #Value (between 0 and 100) to define the percentile above which we cut data with large magErr values
    magErr_mask = (df_obs.magErr <= np.nanpercentile(df_obs.magErr, q=magErr_percentile_cut))
    df_obs = df_obs[magErr_mask]

#Split into previous observations and observations from the most recent night(s)
mask = (df_obs['midPointMjdTai'] < process_mjd-n_new_nights)

df_obs_old = df_obs[mask].copy()
df_obs_new = df_obs[~mask].copy()
logger.info("Previous observations (date < {}): {}".format(process_mjd-1, len(df_obs_old)))
logger.info("New observations ({} <= date < {}): {}".format(process_mjd-1, process_mjd, len(df_obs_new)))

# if len(df_obs_old)<2:
#     #Taken from adler_demo.py
#     logger.info("Insufficient number of previous observations, continuing to next band/object")
#     continue
# if len(df_obs_new)==0:
#     logger.info(f"No new observations in {filt}, continuing to next band/object")
#     continue

#TODO consider how this affects writing out to AdlerData (possibly fine with how it's already setup)
sig_clip_mask = astropy_sigma_clip(df_obs_old[mag_col], sigma=sig_clip_val).mask

df_obs_old = df_obs_old[~sig_clip_mask].copy()

# if len(df_obs_old)<2:
#     #Taken from adler_demo.py
#     logger.info("Insufficient number of previous observations after sigma clipping, continuing to next band/object")
#     continue

median_mag = np.median(df_obs_old[mag_col])
#TODO consider storing relevant res values
res = np.array(df_obs_new[mag_col]) - median_mag

for diff_cut in diff_cuts_arr:
    diff_cut_outlier_arr = sci_utils.outlier_diff(res, diff_cut=diff_cut)
    #Populate outlier rows with True returned by outlier_diff with the current diff_cut value
    df_obs_new.loc[diff_cut_outlier_arr, night_mag_string.format(n_new_nights)] = diff_cut

std_cut_outlier_arr = [False] * len(df_obs_new)
if len(df_obs_old)<4:
    logger.info("Insufficient number of previous observations to check with outlier_sigma_diff")
    #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
else:
    if err_flag:
        logger.info(f"No measurement errors, can't attempt outlier_sigma_diff")
        #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
    else:
        for std_cut in std_cuts_arr:
            std_cut_outlier_arr = sci_utils.outlier_sigma_diff(res, df_obs_new[magErr_col], std_sigma=std_cut)
            df_obs_new.loc[std_cut_outlier_arr, night_std_string.format(n_new_nights)] = std_cut

%matplotlib widget
# Identify any outliers detected
tmp_master_outlier_flag = (diff_cut_outlier_arr) | (std_cut_outlier_arr)
fig, ax = plt.subplots(figsize=(10,6))
ax.errorbar(df_obs_old['midPointMjdTai'], df_obs_old[mag_col], df_obs_old[magErr_col], ls='', marker='.', color='k', label="Previous observations")
ax.errorbar(df_obs_new['midPointMjdTai'], df_obs_new[mag_col], df_obs_new[magErr_col], ls='', marker='.', color='c', label="New observations")
ax.errorbar(df_obs_new[tmp_master_outlier_flag]['midPointMjdTai'], df_obs_new[tmp_master_outlier_flag][mag_col], df_obs_new[tmp_master_outlier_flag][magErr_col], ls='', marker='x', color='b', label="Outliers")
ax.axhline(median_mag)
ax.invert_yaxis()
ax.set_xlabel("Time [MJD]")
ax.set_ylabel(f"{filt}-band Reduced Magnitude")

# Format taking median of new observations and old observations to compare

In [ ]:
# input_rubin_sql_file = f"{root_filepath}rubin_251105.sqlite"
input_rubin_sql_file = f"{root_filepath}lsst-adler/tests/data/mpc_obs_sbn_testing_database.sqlite"

input_conn = sqlite3.connect(input_rubin_sql_file)
input_cur = input_conn.cursor()

# Number of days of data to retrieve (i.e. previous 30 nights)
data_timespan=30

from tqdm import tqdm
import adler.utilities.science_utilities as sci_utils
import math


# Set column to use for magnitude (we don't currently have reduced_mag populated for MPC file format)
mag_col = 'reduced_mag'
magErr_col = 'magErr'

# Set filter list (only ugri present currently, very few u, but keeping in for completeness)
filter_list=['u', 'g', 'r', 'i', 'z', 'y']
# filter_list=['g', 'r', 'i']

make_plots=True

#TODO implement schema switching
schema = "MPC"
# schema = "dp03_catalogs_10yr"

min_obstime_mjd = math.floor(pd.read_sql_query(f"SELECT MIN(mjd_tai) FROM obs_sbn", input_conn).iloc[0].values[0])
max_obstime_mjd = math.ceil(pd.read_sql_query(f"SELECT MAX(mjd_tai) FROM obs_sbn", input_conn).iloc[0].values[0])

In [ ]:
from astropy.stats import sigma_clip as astropy_sigma_clip
sig_clip_val = 3

In [ ]:
# Set thresholds for magnitude changes
diff_cuts_arr = np.array([0, 1, 2, 3]) #0 included due to way we currently write to the output DB
# std_cuts_arr = np.array([3, 4, 5])
std_cuts_arr = np.array([5, 6])

# Defines how many nights to consider as "new observations," allowing for the checking of "sustained outliers/outbursts"
# n_new_nights_arr = np.array([3])

n_new_nights = 3

In [ ]:
# #Defined the strings that will be used as column headers and populated later
# night_mag_string = "outlier_{}night_mag"
# night_std_string = "outlier_{}night_sigma"

# outlier_cols_list = []
# for n_new_nights in n_new_nights_arr:
#     outlier_cols_list.append(night_mag_string.format(n_new_nights))
#     outlier_cols_list.append(night_std_string.format(n_new_nights))

# #TODO include this in function for initialising the output Adler DB
# sql_outlier_cols=""
# for col in outlier_cols_list:
#     sql_outlier_cols += f", {col} INTEGER"

# adler_flags_create_sql = f"CREATE TABLE AdlerSourceFlags(ssObjectId TEXT, filter_name TEXT, diaSourceId TEXT, midPointMjdTai INTEGER{sql_outlier_cols})"

In [ ]:
logger.info("----------------------------------------")
logger.info("New loop started")
logger.info("----------------------------------------")

# process_mjd_arr = np.arange(min_obstime_mjd-0.5, max_obstime_mjd+1.5, 1)
# process_mjd_arr = np.arange(60795.5, 60799.5, 1)
process_mjd_arr = np.array([60799.5])

for process_mjd in process_mjd_arr:
    start_of_night_mjd = process_mjd - 1

    # Get list of objects with observations from most recent night preceding the process date
    # obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE obstime BETWEEN '{start_of_night}' AND '{process_date}' LIMIT 20", input_conn)
    obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE mjd_tai BETWEEN '{start_of_night_mjd}' AND '{process_mjd}'", input_conn)
    unique_obj_ids = obj_df.provid.to_numpy()
    logger.info(f"{len(unique_obj_ids)} objects to analyze")

    if len(unique_obj_ids)==0:
        logger.info(f"No objects to process for {process_mjd}")
        continue

    output_dir = f"outputs_sustained_checks_v2_{process_mjd}"

    os.makedirs(output_dir, exist_ok=True)

    adler_output_filename = f"{output_dir}/adler_output.sqlite"
    conn_adler_out = sqlite3.connect(adler_output_filename)
    
    #TODO tidy this up into function
    cur_adler_out = conn_adler_out.cursor()

    cur_adler_out.execute("DROP TABLE IF EXISTS AdlerData;")
    cur_adler_out.execute("DROP TABLE IF EXISTS  AdlerSourceFlags;")
    
    #TODO make this non-manual
    cur_adler_out.execute(f"CREATE TABLE AdlerData(ssObjectId TEXT, timestamp REAL, g_{n_new_nights}night_sustained, r_{n_new_nights}night_sustained, i_{n_new_nights}night_sustained, u_{n_new_nights}night_sustained, PRIMARY KEY (ssObjectId))")
    # added creation of AdlerSourceFlags table
    # cur_adler_out.execute(adler_flags_create_sql)

    for obj_id in tqdm(unique_obj_ids, desc = f"Objects to process for {process_mjd}"):
        #Taking all data, no time constraint
        planetoid = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssObjectId=obj_id,
                                                            sql_filename=input_rubin_sql_file,
                                                            filter_list=filter_list,
                                                            date_range=[process_mjd-data_timespan, process_mjd])

        adler_data = AdlerData(obj_id, planetoid.filter_list)

        for filt in planetoid.filter_list:
            df_obs = sci_utils.get_df_obs_filt(planetoid, filt=filt)
            
            #Initialise outlier columns in df_obs
            #TODO Consider how we'll load in previous flags if an existing Adler DB is provided
            # for col in outlier_cols_list:
            #     #Populate outlier columns with zeros
            #     df_obs[col] = np.zeros(shape=len(df_obs), dtype=int)
            
            #TODO this may change as no longer checking how many datapoints from previous process
            nobs_nomask = len(df_obs)

            err_flag = df_obs.magErr.isnull().all()
            if err_flag:
                logger.info("All magErr values are NaNs, proceed with caution")
            else:
                # Remove observations with large errorbars
                magErr_percentile_cut = 95    #Value (between 0 and 100) to define the percentile above which we cut data with large magErr values
                magErr_mask = (df_obs.magErr <= np.nanpercentile(df_obs.magErr, q=magErr_percentile_cut))
                df_obs = df_obs[magErr_mask]
            
            # for n_new_nights in n_new_nights_arr:
            #Split into previous observations and observations from the most recent night(s)
            mask = (df_obs['midPointMjdTai'] < process_mjd-n_new_nights)
            
            #TODO update the splitting and checking of previous nights (at least 2 nights in previous 3-5 nights to do sustained check
            df_obs_old = df_obs[mask].copy()
            df_obs_new = df_obs[~mask].copy()
            logger.info("Previous observations (date < {}): {}".format(process_mjd-1, len(df_obs_old)))
            logger.info("New observations ({} <= date < {}): {}".format(process_mjd-1, process_mjd, len(df_obs_new)))

            if len(df_obs_old)<2:
                # Taken from adler_demo.py
                logger.info("Insufficient number of previous observations, continuing to next band/object")
                continue
            if len(df_obs_new)==0:
                logger.info(f"No new observations in {filt}, continuing to next band/object")
                continue

            df_obs_new.sort_values(by="midPointMjdTai", inplace=True)

            time_gaps = sci_utils.apparition_gap_finder(df_obs_new.midPointMjdTai.to_numpy(),  dx=0.5)
            if len(time_gaps)==0:
                #If there is only one night of new data, we continue to the next band/object
                logger.info("Insufficient number of nights with new observations, continuing to next band/object")
                continue

            #TODO consider how this affects writing out to AdlerData (possibly fine with how it's already setup)
            sig_clip_mask = astropy_sigma_clip(df_obs_old[mag_col], sigma=sig_clip_val).mask

            df_obs_old = df_obs_old[~sig_clip_mask].copy()

            sig_clip_mask = astropy_sigma_clip(df_obs_new[mag_col], sigma=sig_clip_val).mask

            df_obs_new = df_obs_new[~sig_clip_mask].copy()

            if len(df_obs_old)<2:
                # Taken from adler_demo.py
                logger.info("Insufficient number of previous observations after sigma clipping, continuing to next band/object")
                continue
            if len(df_obs_new)==0:
                logger.info(f"No new observations in {filt} after sigma clipping, continuing to next band/object")
                continue

            old_median_mag = np.median(df_obs_old[mag_col])
            
            new_median_mag = np.median(df_obs_new[mag_col])

            # #TODO consider storing relevant res values
            # res = np.array(df_obs_new[mag_col]) - median_mag

            diff_cut_threshold_arr = np.abs(new_median_mag - old_median_mag) > diff_cuts_arr

            diff_cut_met = np.max(diff_cuts_arr[diff_cut_threshold_arr])

            
            
            #TODO add columns to adlerdata (temporary version) to store these sustaine doutliers, fix the plot down below

            # for diff_cut in diff_cuts_arr:
            #     diff_cut_outlier_arr = sci_utils.outlier_diff(res, diff_cut=diff_cut)
            #     #Populate outlier rows with True returned by outlier_diff with the current diff_cut value
            #     df_obs_new.loc[diff_cut_outlier_arr, night_mag_string.format(n_new_nights)] = diff_cut
            
            # std_cut_outlier_arr = [False] * len(df_obs_new)
            # if len(df_obs_old)<4:
            #     logger.info("Insufficient number of previous observations to check with outlier_sigma_diff")
            #     #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
            # else:
            #     if err_flag:
            #         logger.info(f"No measurement errors, can't attempt outlier_sigma_diff")
            #         #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
            #     else:
            #         for std_cut in std_cuts_arr:
            #             std_cut_outlier_arr = sci_utils.outlier_sigma_diff(res, df_obs_new[magErr_col], std_sigma=std_cut)
            #             df_obs_new.loc[std_cut_outlier_arr, night_std_string.format(n_new_nights)] = std_cut
            
            if make_plots:
                # Identify any outliers detected
                fig, ax = plt.subplots()
                ax.errorbar(df_obs_old['midPointMjdTai'], df_obs_old[mag_col], df_obs_old[magErr_col], ls='', marker='.', color='k', label="Previous observations")
                ax.errorbar(df_obs_new['midPointMjdTai'], df_obs_new[mag_col], df_obs_new[magErr_col], ls='', marker='.', color='c', label="New observations")
                ax.axhline(old_median_mag, ls='--', c='k')
                ax.axhline(new_median_mag, ls='--', c='c')
                ax.invert_yaxis()
                ax.set_xlabel("Time [MJD]")
                ax.set_ylabel(f"{filt}-band Reduced Magnitude")
                fig.savefig(f"{output_dir}/{obj_id}_{filt}_{n_new_nights}nights_outliers.png", bbox_inches='tight', pad_inches=0.05)
                plt.close(fig)

                # #Use pandas.DataFrame.update to add the outlier flags to the df_obs DataFrame
                # df_obs.update(df_obs_new)

            # Identify rows that contain at least 1 outlier flag set to a value greater than 0
            # master_outlier_flag = df_obs[outlier_cols_list].any(axis=1)
            # df_obs.loc[master_outlier_flag,['ssObjectId', 'filter_name', 'diaSourceId', 'midPointMjdTai'] + outlier_cols_list].to_sql('AdlerSourceFlags', con=conn_adler_out, if_exists='append', index=False)
            
            #TODO consider how to include check and flag for if outliers span multiple consecutive nights
            #Perhaps this is easier now we update df_obs
            #Some kind of resetting cumulative sum to test for sustained outliers that then looks for any value above say 3. [0,1,0,1,2,0] gives [0,1,0,1,3,0]

            #Write nobs to AdlerData
            #TODO write median and standard deviation of previous observations to AdlerData
            #Create AdlerData structure to do the above where we have a particular model that is designed for the median
            adler_data.populate_phase_parameters(filt, **{'nobs':nobs_nomask})
            adler_data.write_row_to_database(adler_output_filename)

            cur_adler_out.execute(f"UPDATE AdlerData SET {filt}_{n_new_nights}night_sustained = {diff_cut_met} WHERE ssObjectId='{obj_id}'")
            conn_adler_out.commit()

            logger.info(f"New information for {obj_id} written to AdlerSourceFlags and AdlerData tables")

    #TODO make summary stats a function
    # logger.info(f"Computing summary statistics for {process_mjd}")
    
    # cur_adler_out.execute("SELECT COUNT(*) FROM AdlerData")
    # n_obj_analzyed = cur_adler_out.fetchall()[0][0]
    # logger.info(f"Number of objects analysed: {n_obj_analzyed}")

    # # Get column information to check what filters have previously been analysed (and therefore have {filter}_nobs columns in AdlerData)
    # cur_adler_out.execute("PRAGMA table_info(AdlerData);")
    # adler_out_cols = [row[1] for row in cur_adler_out.fetchall()]
    # filter_nobs_columns = [c for c in adler_out_cols if c.endswith('_nobs')]
    # current_adlerdata_filters = [c.replace('_nobs', '') for c in filter_nobs_columns]

    # # #Manually removing u filter because it has so few observations in comparison
    # # filter_nobs_columns.remove('u_nobs')

    # filter_columns_sql = ', '.join(filter_nobs_columns)
    # nobs_df = pd.read_sql_query(f"SELECT ssObjectId, {filter_columns_sql} FROM AdlerData", conn_adler_out)

    # for band in current_adlerdata_filters:
    #     med_obs_band = np.median(nobs_df[f"{band}_nobs"])
    #     std_obs_band = np.std(nobs_df[f"{band}_nobs"])
    #     sum_obs_band = np.sum(nobs_df[f"{band}_nobs"])
        
    #     logger.info(f"Total number of observations for {band}-band: {sum_obs_band}")
    #     logger.info(f"Observations per object for {band}-band: Median={med_obs_band}, standard deviation={std_obs_band}")
        

    # filters_to_plot = ['g', 'r', 'i']
    # fig, ax= plt.subplots()
    # ax.hist((nobs_df['g_nobs'], nobs_df['r_nobs'], nobs_df['i_nobs']),
    #         color=[plot_filter_colors_white_background[filt] for filt in filters_to_plot],
    #         label=[f"{filt}-band" for filt in filters_to_plot])

    # ax.set_xlabel("Observations per filter")
    # ax.set_ylabel("Number of objects")
    # ax.legend()
    # fig.savefig(f"{output_dir}/obs_per_band_hists.png", bbox_inches='tight', pad_inches=0.05)
    # plt.close(fig)
    # logger.info(f"Histograms of observations per band per object saved to {output_dir}/obs_per_band_hists.png")
    
    # for n_new_nights in n_new_nights_arr:
    #     logger.info(f"Checking for number of outliers when considering last {n_new_nights} night(s) as new observations")
    #     for diff_cut in diff_cuts_arr:
    #         cur_adler_out.execute(f"SELECT COUNT(*) FROM AdlerSourceFlags WHERE {night_mag_string.format(n_new_nights)}>={diff_cut}")
    #         logger.info(f"Number of outliers above {diff_cut} magnitude: {cur_adler_out.fetchall()[0][0]}")
    #     for std_cut in std_cuts_arr:
    #         cur_adler_out.execute(f"SELECT COUNT(*) FROM AdlerSourceFlags WHERE {night_std_string.format(n_new_nights)}>={std_cut}")
    #         logger.info(f"Number of outliers above {std_cut}-sigma: {cur_adler_out.fetchall()[0][0]}")

    # #TODO perhaps add example query to find the outliers in the output database

    # #TODO distribution of outliers per object and per filter

# Trying to use new class structures

In [ ]:
logger.info("----------------------------------------")
logger.info("New loop started")
logger.info("----------------------------------------")

# process_mjd_arr = np.arange(min_obstime_mjd-0.5, max_obstime_mjd+1.5, 1)
# process_mjd_arr = np.arange(60795.5, 60799.5, 1)
process_mjd_arr = np.array([60799.5])

for process_mjd in process_mjd_arr:
    process_date = mjd_to_utc(process_mjd)
    start_of_night = mjd_to_utc(process_mjd - 1)

    # Get list of objects with observations from most recent night preceding the process date
    # obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE obstime BETWEEN '{start_of_night}' AND '{process_date}' LIMIT 20", input_conn)
    obj_df = pd.read_sql_query(f"SELECT DISTINCT provid FROM obs_sbn WHERE obstime BETWEEN '{start_of_night}' AND '{process_date}'", input_conn)
    unique_obj_ids = obj_df.provid.to_numpy()
    logger.info(f"{len(unique_obj_ids)} objects to analyze")

    if len(unique_obj_ids)==0:
        logger.info(f"No objects to process for {process_mjd}")
        continue

    output_dir = f"outputs_lesscols_update_full_{process_mjd}"

    os.makedirs(output_dir, exist_ok=True)

    adler_output_filename = f"{output_dir}/adler_output.sqlite"
    conn_adler_out = sqlite3.connect(adler_output_filename)
    
    #TODO tidy this up into function
    cur_adler_out = conn_adler_out.cursor()

    cur_adler_out.execute("DROP TABLE IF EXISTS AdlerData;")
    cur_adler_out.execute("DROP TABLE IF EXISTS  AdlerSourceFlags;")

    cur_adler_out.execute("CREATE TABLE AdlerData(ssObjectId TEXT, timestamp REAL, PRIMARY KEY (ssObjectId))")
    # added creation of AdlerSourceFlags table
    cur_adler_out.execute(adler_flags_create_sql)

    for obj_id in tqdm(unique_obj_ids, desc = f"Objects to process for {process_mjd}"):
        #Taking all data, no time constraint
        planetoid = AdlerPlanetoid.construct_from_mpc_obs_sbn(ssObjectId=obj_id,
                                                            sql_filename=input_rubin_sql_file,
                                                            filter_list=filter_list,
                                                            date_range=[process_mjd-data_timespan, process_mjd])

        adler_data = AdlerData(obj_id, planetoid.filter_list)

        for filt in planetoid.filter_list:
            df_obs = sci_utils.get_df_obs_filt(planetoid, filt=filt)
            
            #Initialise outlier columns in df_obs
            #TODO Consider how we'll load in previous flags if an existing Adler DB is provided
            for col in outlier_cols_list:
                #Populate outlier columns with zeros
                df_obs[col] = np.zeros(shape=len(df_obs), dtype=int)
            
            #TODO this may change as no longer checking how many datapoints from previous process
            nobs_nomask = len(df_obs)

            err_flag = df_obs.magErr.isnull().all()
            if err_flag:
                logger.info("All magErr values are NaNs, proceed with caution")
            else:
                # Remove observations with large errorbars
                magErr_percentile_cut = 95    #Value (between 0 and 100) to define the percentile above which we cut data with large magErr values
                magErr_mask = (df_obs.magErr <= np.nanpercentile(df_obs.magErr, q=magErr_percentile_cut))
                df_obs = df_obs[magErr_mask]
            
            for n_new_nights in n_new_nights_arr:
                #Split into previous observations and observations from the most recent night(s)
                mask = (df_obs['midPointMjdTai'] < process_mjd-n_new_nights)

                df_obs_old = df_obs[mask].copy()
                df_obs_new = df_obs[~mask].copy()
                logger.info("Previous observations (date < {}): {}".format(process_mjd-1, len(df_obs_old)))
                logger.info("New observations ({} <= date < {}): {}".format(process_mjd-1, process_mjd, len(df_obs_new)))

                if len(df_obs_old)<2:
                    # Taken from adler_demo.py
                    logger.info("Insufficient number of previous observations, continuing to next band/object")
                    continue
                if len(df_obs_new)==0:
                    logger.info(f"No new observations in {filt}, continuing to next band/object")
                    continue

                #TODO consider how this affects writing out to AdlerData (possibly fine with how it's already setup)
                sig_clip_mask = astropy_sigma_clip(df_obs_old[mag_col], sigma=sig_clip_val).mask

                df_obs_old = df_obs_old[~sig_clip_mask].copy()

                if len(df_obs_old)<2:
                    # Taken from adler_demo.py
                    logger.info("Insufficient number of previous observations after sigma clipping, continuing to next band/object")
                    continue

                median_mag = np.median(df_obs_old[mag_col])
                #TODO consider storing relevant res values
                res = np.array(df_obs_new[mag_col]) - median_mag
                
                for diff_cut in diff_cuts_arr:
                    diff_cut_outlier_arr = sci_utils.outlier_diff(res, diff_cut=diff_cut)
                    #Populate outlier rows with True returned by outlier_diff with the current diff_cut value
                    df_obs_new.loc[diff_cut_outlier_arr, night_mag_string.format(n_new_nights)] = diff_cut
                
                std_cut_outlier_arr = [False] * len(df_obs_new)
                if len(df_obs_old)<4:
                    logger.info("Insufficient number of previous observations to check with outlier_sigma_diff")
                    #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
                else:
                    if err_flag:
                        logger.info(f"No measurement errors, can't attempt outlier_sigma_diff")
                        #TODO how to handle this case (i.e. do we populate the columns with something other than False?)
                    else:
                        for std_cut in std_cuts_arr:
                            std_cut_outlier_arr = sci_utils.outlier_sigma_diff(res, df_obs_new[magErr_col], std_sigma=std_cut)
                            df_obs_new.loc[std_cut_outlier_arr, night_std_string.format(n_new_nights)] = std_cut
                
                if make_plots:
                    # Identify any outliers detected
                    tmp_master_outlier_flag = (diff_cut_outlier_arr) | (std_cut_outlier_arr)
                    fig, ax = plt.subplots()
                    ax.errorbar(df_obs_old['midPointMjdTai'], df_obs_old[mag_col], df_obs_old[magErr_col], ls='', marker='.', color='k', label="Previous observations")
                    ax.errorbar(df_obs_new['midPointMjdTai'], df_obs_new[mag_col], df_obs_new[magErr_col], ls='', marker='.', color='c', label="New observations")
                    ax.errorbar(df_obs_new[tmp_master_outlier_flag]['midPointMjdTai'], df_obs_new[tmp_master_outlier_flag][mag_col], df_obs_new[tmp_master_outlier_flag][magErr_col], ls='', marker='x', color='b', label="Outliers")
                    ax.axhline(median_mag)
                    ax.invert_yaxis()
                    ax.set_xlabel("Time [MJD]")
                    ax.set_ylabel(f"{filt}-band Reduced Magnitude")
                    fig.savefig(f"{output_dir}/{obj_id}_{filt}_{n_new_nights}nights_outliers.png", bbox_inches='tight', pad_inches=0.05)
                    plt.close(fig)

                #Use pandas.DataFrame.update to add the outlier flags to the df_obs DataFrame
                df_obs.update(df_obs_new)

            # Identify rows that contain at least 1 outlier flag set to a value greater than 0
            master_outlier_flag = df_obs[outlier_cols_list].any(axis=1)
            df_obs.loc[master_outlier_flag,['ssObjectId', 'filter_name', 'diaSourceId', 'midPointMjdTai'] + outlier_cols_list].to_sql('AdlerSourceFlags', con=conn_adler_out, if_exists='append', index=False)
            
            #TODO consider how to include check and flag for if outliers span multiple consecutive nights
            #Perhaps this is easier now we update df_obs
            #Some kind of resetting cumulative sum to test for sustained outliers that then looks for any value above say 3. [0,1,0,1,2,0] gives [0,1,0,1,3,0]

            #Write nobs to AdlerData
            #TODO write median and standard deviation of previous observations to AdlerData
            #Create AdlerData structure to do the above where we have a particular model that is designed for the median
            adler_data.populate_phase_parameters(filt, **{'nobs':nobs_nomask})
            adler_data.write_row_to_database(adler_output_filename)

            logger.info(f"New information for {obj_id} written to AdlerSourceFlags and AdlerData tables")

    #TODO make summary stats a function
    logger.info(f"Computing summary statistics for {process_mjd}")
    
    cur_adler_out.execute("SELECT COUNT(*) FROM AdlerData")
    n_obj_analzyed = cur_adler_out.fetchall()[0][0]
    logger.info(f"Number of objects analysed: {n_obj_analzyed}")

    # Get column information to check what filters have previously been analysed (and therefore have {filter}_nobs columns in AdlerData)
    cur_adler_out.execute("PRAGMA table_info(AdlerData);")
    adler_out_cols = [row[1] for row in cur_adler_out.fetchall()]
    filter_nobs_columns = [c for c in adler_out_cols if c.endswith('_nobs')]
    current_adlerdata_filters = [c.replace('_nobs', '') for c in filter_nobs_columns]

    # #Manually removing u filter because it has so few observations in comparison
    # filter_nobs_columns.remove('u_nobs')

    filter_columns_sql = ', '.join(filter_nobs_columns)
    nobs_df = pd.read_sql_query(f"SELECT ssObjectId, {filter_columns_sql} FROM AdlerData", conn_adler_out)

    for band in current_adlerdata_filters:
        med_obs_band = np.median(nobs_df[f"{band}_nobs"])
        std_obs_band = np.std(nobs_df[f"{band}_nobs"])
        sum_obs_band = np.sum(nobs_df[f"{band}_nobs"])
        
        logger.info(f"Total number of observations for {band}-band: {sum_obs_band}")
        logger.info(f"Observations per object for {band}-band: Median={med_obs_band}, standard deviation={std_obs_band}")
        

    filters_to_plot = ['g', 'r', 'i']
    fig, ax= plt.subplots()
    ax.hist((nobs_df['g_nobs'], nobs_df['r_nobs'], nobs_df['i_nobs']),
            color=[plot_filter_colors_white_background[filt] for filt in filters_to_plot],
            label=[f"{filt}-band" for filt in filters_to_plot])

    ax.set_xlabel("Observations per filter")
    ax.set_ylabel("Number of objects")
    ax.legend()
    fig.savefig(f"{output_dir}/obs_per_band_hists.png", bbox_inches='tight', pad_inches=0.05)
    plt.close(fig)
    logger.info(f"Histograms of observations per band per object saved to {output_dir}/obs_per_band_hists.png")
    
    for n_new_nights in n_new_nights_arr:
        logger.info(f"Checking for number of outliers when considering last {n_new_nights} night(s) as new observations")
        for diff_cut in diff_cuts_arr:
            cur_adler_out.execute(f"SELECT COUNT(*) FROM AdlerSourceFlags WHERE {night_mag_string.format(n_new_nights)}>={diff_cut}")
            logger.info(f"Number of outliers above {diff_cut} magnitude: {cur_adler_out.fetchall()[0][0]}")
        for std_cut in std_cuts_arr:
            cur_adler_out.execute(f"SELECT COUNT(*) FROM AdlerSourceFlags WHERE {night_std_string.format(n_new_nights)}>={std_cut}")
            logger.info(f"Number of outliers above {std_cut}-sigma: {cur_adler_out.fetchall()[0][0]}")

    #TODO perhaps add example query to find the outliers in the output database

    #TODO distribution of outliers per object and per filter